# INF221 Assignment 3
Jon Nordby

In [6]:
%matplotlib inline

import math
import random

import matplotlib
import pandas

## Exercise 10

CLRS 4.1-1: What does Find-Maximum-Subarray return when all elements of A are negative?


zero / empty-set ? 

## Exercise 11

CLRS 4.1-3 (partial):

Implement both the brute-force and the recursive algorithms for the maximum-subarray problem on your own computer using Python. What problem size $n_0$ gives the crossover point at which the recursive algorithm beats the brute force algorithm?

Hints:

Use Python module timeit to time execution of your code. 
To create input data, use cumulative sums of random numbers as in the example towards the end of Lecture 4.


In [2]:
# page 68-74
def maximum_subarray_bruteforce(A):
    pass


def maximum_subarray_recursive(A):
    pass



## Exercise 12

CLRS 4.3-1 & 2:

a) Show that the solution of $T(n)=T(n-1)+n$ is $O(n^2)$.

b) Show that the solution of $T(n) = T(\lceil n/2 \rceil)+1$ is $O(\lg n)$.

 Recursion
 Substitution method?
 See Assignment 2, Ex 7

## Exercise 13
CLRS 4.3-3

We saw that the solution of $T(n)=2T(\lfloor n/2 \rfloor)+n$ is $O(n\lg n)$.
Show that the solution of this recurrence is also: $\Omega(n\lg n)$ and conclude that the solution is $\Theta(n\lg n)$.

Hint: Use the same approach as for the upper bound in Ch 4.3. You will then need to find at some point an expression that is smaller than $\lfloor n/2 \rfloor \lg(\lfloor n/2 \rfloor)$. Here, it can be useful to use different lower bounds for $\lfloor n/2 \rfloor$ outside and inside the logarithm.


## Exercise 14
CLRS 4.4-1, 5:

Use recursion trees to determine a good asymptotic upper bound on the recurrences given below and then use the substitution method to verify your answers.

Note: This exercise can give you up to two points extra credit. You receive one point each for solving a) and b), and a third point for a tight bound in b).

a) $T(n) = 3T(\lfloor n/2\rfloor)+n$

Hint: The recursion tree analysis analysis is similar to the first example in Ch 4.4. For the proof using the substitution method, also take a look at the "Subtleties" section of Ch 4.3.

b) $T(n) = T(n-1)+T(n/2) + nT(n)$

Hint: To obtain an upper bound, perform a "worst case" analysis along the left edge of the recursion tree and assume that all levels are full. Prove that bound using the substitution method. For extra credit, attempt at tighter bound!

Page 86-90

## Exercise 15
CLRS 4.5-1:

Use the master method to give tight asymptotic bounds for the following recurrences:

a) $T(n) = 2T(n/4)+1$

b) $T(n) = 2T(n/4) + \sqrt{n}$

c) $T(n) = 2T(n/4)+nT(n)$

d) $T(n) = 2T(n/4)+n^2$

Page 93 - 96
Check that the method applies (possibly rearranging T ?)

$T(n) = aT(n/b) + f(n)$ where $a \geq 1$ and $b \gt 1$ and $f(n)$ is asymptotically positive function.

Dermine which case it falls into

1. If $ f(n) = O(n^{\log_b a-e}) $ for some constant $e > 0$, then $T(n) = \Theta(n^{\log_b a})$.

2. If $f(n) = \Theta(n^{\log_b a})$ , then $ T(n) = \Theta(n^{\log_b a} \lg n)$.

3. If $ f(n) = \Omega(n^{\log_b a+e})$ for some constant $e > 0$, and if $af(n/b) \leq cf(n)$ for
some constant $c < 1$ and all sufficiently large $n$, then $T(n) = \Theta(f(n))$.

Note, in case 1 and 3 $f(n)$must be *polynomally* smaller/larger than $n^{\log_b a-e}$, that is, asymptotcally larger by a factor $n^e$ for some constant $e \gt 0$

Calculate